<a href="https://colab.research.google.com/github/AmlanSamanta/TensorFlow_Examples/blob/main/Model_Saving_and_loading_basics_with_tf_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Saving and loading a model or model weights during and after the training process are the fundamental necessities of the developer community when somebody needs to work on a ml project and/or share the model creation code and trained model weights/parameters with others for reproduction of their work or checking with different datasets to understand how their models work.

The Keras API helps us to store the model weights to a collection of checkpoint-formatted files that contain only the trained weights in a binary format. Checkpoints contain:

    One or more shards that contain your model's weights.
    An index file that indicates which weights are stored in which shard.


my installs and Imports

In [20]:
!pip install h5py
import os

import tensorflow as tf
from tensorflow import keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Getting my example mnist dataset - using first 1000 examples




In [21]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

Defining and building my model

In [22]:
# a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# creating a basic model instance
model = create_model()

# displaying model's architecture
model.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Saving model checkpoints during my model training using Keras callback

In [25]:
checkpoint_path = "training_310123_0744_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# creating the callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# training my model along with passing the callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])


Epoch 1/10
23/32 [====================>.........] - ETA: 0s - loss: 0.0344 - sparse_categorical_accuracy: 0.9986
Epoch 1: saving model to training_310123_0744_1/cp.ckpt
32/32 [==============================] - 1s 9ms/step - loss: 0.0352 - sparse_categorical_accuracy: 0.9990 - val_loss: 0.3864 - val_sparse_categorical_accuracy: 0.8790
Epoch 2/10
23/32 [====================>.........] - ETA: 0s - loss: 0.0255 - sparse_categorical_accuracy: 0.9986
Epoch 2: saving model to training_310123_0744_1/cp.ckpt
32/32 [==============================] - 0s 11ms/step - loss: 0.0278 - sparse_categorical_accuracy: 0.9990 - val_loss: 0.3949 - val_sparse_categorical_accuracy: 0.8730
Epoch 3/10
22/32 [===================>..........] - ETA: 0s - loss: 0.0266 - sparse_categorical_accuracy: 0.9986
Epoch 3: saving model to training_310123_0744_1/cp.ckpt
32/32 [==============================] - 0s 11ms/step - loss: 0.0248 - sparse_categorical_accuracy: 0.9990 - val_loss: 0.3963 - val_sparse_categorical_accurac

In [26]:
# at the end of each epoch this single collection of TF checkpoint files are updated
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.index', 'cp.ckpt.data-00000-of-00001']

Loading my model's weights from the saved checkpoints  

In [27]:
# creating another fresh model instance with same architecture
model = create_model()

# evaluate the model before trining
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("untrained model, accuracy: {:5.2f}%".format(100 * acc))


# restoring previous model by loading the weights from checkpoints
model.load_weights(checkpoint_path)

# re-evaluating the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3415 - sparse_categorical_accuracy: 0.1070 - 167ms/epoch - 5ms/step
untrained model, accuracy: 10.70%
32/32 - 0s - loss: 0.4169 - sparse_categorical_accuracy: 0.8680 - 176ms/epoch - 5ms/step
restored model, accuracy: 86.80%


Utilizing callback options - saving my checkpoints once every five epoch with unique name

In [28]:
# including the epoch in the file name by using `str.format`
checkpoint_path = "training_310123_0747_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# creating a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)


model = create_model()

# saving the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# training the model and saving the uniquely named checkpoints every 5 epochs with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)



Epoch 5: saving model to training_310123_0747_2/cp-0005.ckpt

Epoch 10: saving model to training_310123_0747_2/cp-0010.ckpt

Epoch 15: saving model to training_310123_0747_2/cp-0015.ckpt

Epoch 20: saving model to training_310123_0747_2/cp-0020.ckpt

Epoch 25: saving model to training_310123_0747_2/cp-0025.ckpt

Epoch 30: saving model to training_310123_0747_2/cp-0030.ckpt

Epoch 35: saving model to training_310123_0747_2/cp-0035.ckpt

Epoch 40: saving model to training_310123_0747_2/cp-0040.ckpt

Epoch 45: saving model to training_310123_0747_2/cp-0045.ckpt

Epoch 50: saving model to training_310123_0747_2/cp-0050.ckpt


In [29]:
os.listdir(checkpoint_dir)

['cp-0050.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0010.ckpt.index']

In [9]:
# Choosing the latest checkpoint
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_310523_0747_2/cp-0050.ckpt'

In [30]:
# testing with the latest checkpoint


model = create_model()


model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4818 - sparse_categorical_accuracy: 0.8770 - 177ms/epoch - 6ms/step
Restored model, accuracy: 87.70%


Saving weights manually

In [32]:

model.save_weights('./checkpoints/my_checkpoint')

model = create_model()

model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))


32/32 - 0s - loss: 0.4818 - sparse_categorical_accuracy: 0.8770 - 173ms/epoch - 5ms/step
Restored model, accuracy: 87.70%


Saving my entire model including its architecture, weights and training configuration usiing SavedModel format

In [33]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5


32/32 [==============================] - 1s 4ms/step - loss: 1.1761 - sparse_categorical_accuracy: 0.6450
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4207 - sparse_categorical_accuracy: 0.8780
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2894 - sparse_categorical_accuracy: 0.9200
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2086 - sparse_categorical_accuracy: 0.9510
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1466 - sparse_categorical_accuracy: 0.9720


In [34]:
# Inspecting my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model


my_model
assets	keras_metadata.pb  saved_model.pb  variables


In [35]:
#Restoring the saved model
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()


Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 512)               401920    
                                                                 
 dropout_14 (Dropout)        (None, 512)               0         
                                                                 
 dense_29 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Evaluating the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)


32/32 - 0s - loss: 0.4365 - sparse_categorical_accuracy: 0.8580 - 173ms/epoch - 5ms/step
Restored model, accuracy: 85.80%
32/32 [==============================] - 0s 2ms/step
(1000, 10)


Saving using HDF5 format

In [17]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')


Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.1522 - sparse_categorical_accuracy: 0.6780
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4154 - sparse_categorical_accuracy: 0.8890
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2822 - sparse_categorical_accuracy: 0.9150
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2038 - sparse_categorical_accuracy: 0.9500
Epoch 5/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1556 - sparse_categorical_accuracy: 0.9650


In [18]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [19]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))


32/32 - 0s - loss: 0.4598 - sparse_categorical_accuracy: 0.8470 - 182ms/epoch - 6ms/step
Restored model, accuracy: 84.70%
